<a href="https://colab.research.google.com/github/vdslab/t-taiki0620/blob/main/Okapi_BM25_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install unidic-lite
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
!pip install mojimoji

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl libio-string

In [2]:
import os
import json
import csv
import pandas as pd
import MeCab
import subprocess
import mojimoji

path = '/content/drive/MyDrive/卒業演習/tsne_+_clusters_list.json'

with open(path, encoding='utf8')as f:
  df = pd.read_json(f)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22115 entries, 0 to 22114
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   22115 non-null  int64  
 1   プロジェクトID             22115 non-null  int64  
 2   公開年度                 22115 non-null  int64  
 3   事業番号1                22115 non-null  object 
 4   事業番号2                22115 non-null  int64  
 5   事業番号3                150 non-null    float64
 6   主要政策・施策              22115 non-null  object 
 7   府省庁                  22115 non-null  object 
 8   事業名                  22115 non-null  object 
 9   事業の目的                22115 non-null  object 
 10  事業概要                 22115 non-null  object 
 11  執行額                  22115 non-null  float64
 12  objective_x          22115 non-null  float64
 13  objective_y          22115 non-null  float64
 14  cluster_in_ministry  22115 non-null  int64  
 15  ALL                  22115 non-null 

In [3]:
df

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y,cluster_in_ministry,ALL,高齢社会対策,男女共同参画,少子化社会対策,子ども・若者育成支援,国土強靱化施策,障害者施策,地方創生,科学技術・イノベーション,ＩＴ戦略,一億総活躍推進,海洋政策,観光立国,宇宙開発利用,地球温暖化対策,ＯＤＡ,自殺対策,医療分野の研究開発関連,沖縄振興,知的財産,2020年東京オリパラ,犯罪被害者等施策,クールジャパン,食育推進,その他
0,13,16,2015,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、迅速・正確に内外に分かりやすく伝え...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,123.00,-3.427058,36.508163,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10604,16,2017,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,120.00,-3.443014,36.496967,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,24982,16,2019,nan,21,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,106.00,-3.433842,36.497105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.00,-3.439199,36.503059,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,5377,16,2016,nan,12,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、迅速・正確に内外に分かりやすく伝え...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,85.00,-3.429540,36.508717,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22110,22976,8983,2019,nan,117,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1628.00,-0.105321,-19.544006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22111,25938,44,2019,nan,150,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,30.73,28.377846,-26.652079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22112,25500,4895,2019,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,① 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7282.00,-7.667183,-13.079095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22113,13514,4916,2017,nan,38,NaN,['地方創生'],農林水産省,農林水産物の品目別輸出促進緊急対策事業,「総合的なＴＰＰ関連政策大綱」においては、高品質な我が国農林水産物の一層の輸出拡大により、「...,「農林水産業の輸出力強化戦略」及び「日本再興戦略2016」に基づき、コメ、青果物、花き、茶、...,431.00,-43.182682,-32.702126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(columns=['高齢社会対策', '男女共同参画', '少子化社会対策', '子ども・若者育成支援', '国土強靱化施策', '障害者施策', '地方創生', '科学技術・イノベーション', 'ＩＴ戦略', '一億総活躍推進', '海洋政策', '観光立国', '宇宙開発利用', '地球温暖化対策', 'ＯＤＡ', '自殺対策', '医療分野の研究開発関連', '沖縄振興', '知的財産', '2020年東京オリパラ', '犯罪被害者等施策', 'クールジャパン', '食育推進', 'その他'])

In [5]:
df = df.drop(columns=['cluster_in_ministry', 'ALL'])

In [6]:
df_2018 = df.iloc[0:0]
print(df_2018)
cnt = 0
for i in range(len(df)):
  if df.loc[i]['公開年度'] == 2018:
    df_2018.loc[cnt] = df.loc[i]
    cnt += 1

Empty DataFrame
Columns: [ID, プロジェクトID, 公開年度, 事業番号1, 事業番号2, 事業番号3, 主要政策・施策, 府省庁, 事業名, 事業の目的, 事業概要, 執行額, objective_x, objective_y]
Index: []


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
df_2018

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y
0,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.0,-3.439199,36.503059
1,15755,15,2018,nan,1,NaN,['その他'],内閣官房,内閣人事局経費,政府全体を通じた成果向上及び人材育成を狙いとして実施する。\n各研修に応じ、①幹部候補育成課...,１．官民合同セミナー（幹部級）・(課長級)（幹部級：本府省の幹部職員並びに民間企業の取締役ク...,23.0,57.182091,-2.059250
2,15926,52,2018,nan,124,NaN,['その他'],内閣府,人道救援物資備蓄経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合等から人道的な国際救援活動への物資協力の要請をうけた場合、迅速に対応できるよう人道救...,59.0,46.140083,-57.230614
3,15964,35,2018,nan,162,NaN,['クールジャパン'],内閣府,クールジャパン戦略推進経費,ゲーム・マンガ・アニメといったコンテンツ、ファッション、産品、日本食、伝統文化、デザイン等、...,クールジャパン担当大臣等の海外出張を活用したクールジャパン発信イベント、クールジャパン戦略の...,49.0,2.651179,-55.060982
4,15924,84,2018,nan,122,NaN,['その他'],内閣府,国際平和協力隊の派遣等経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合の要請に基づき、引き続き南スーダンにおいて、国際平和協力業務を実施する。,78.0,46.272320,-57.395348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4368,20146,3574,2018,nan,32,NaN,['地球温暖化対策'],環境省,短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金関連業務,国内におけるブラックカーボン等の短期寿命気候汚染物質（SLCP）の排出実態及び対策技術等に係...,１．短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金\n CCACに対し、平成24...,327.0,-10.464338,-70.462219
4369,15941,44,2018,nan,139,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,21.6,28.345156,-26.662083
4370,18840,4895,2018,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,１ 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7886.0,-7.659595,-13.079552
4371,19154,3708,2018,nan,119,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1454.0,-0.068404,-19.573669


In [8]:
import re
def normalize_text(text):
    return mojimoji.zen_to_han(text, kana = False).replace(' ', '').lower().split()

In [65]:
import re
def normalize_text(text):
  # 全部全角に変換
  result = mojimoji.han_to_zen(text)

  # 全角から半角に変換（カナは除く）
  result = mojimoji.zen_to_han(result, kana=False)

  # 全ての文字を小文字に変換
  result = result.lower()
  
  return result
  # return re.sub(' ', '', mojimoji.zen_to_han(text)).upper().split()

In [49]:
# 0404 used this
cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
def tokenize(text):
  m=MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
  lines = m.parse(text).splitlines()
  words = []
  for line in lines:
    chunks = line.split('\t')
    # print(chunks[0])
    if chunks[0] != 'EOS' and not chunks[1].startswith('記号'):
      words.append(chunks[0])
  return words

In [42]:
from gensim.models import word2vec
model   = word2vec.Word2Vec.load("/content/drive/MyDrive/Colab Notebooks/pre-judgit/ja-MeCab-50.data.model")

In [73]:
tf_business_list = []
word_list = []
for i in range(len(df_2018)):
  tf_business_list.append(df_2018.loc[i])
  if df_2018.loc[i]['事業概要'] != '':
    if normalize_text(df_2018.loc[i]['事業概要']) != []:
      print(i, df_2018.loc[i]['事業概要'])
      words = tokenize(normalize_text(df_2018.loc[i]['事業概要']))
      for word in words:
        if word not in word_list:
          word_list.append(word)

ストリーミング出力は最後の 5000 行に切り捨てられました。
2690 　土地改良事業等の農家負担金の軽減を図るため、
・地域の中心となる経営体に一定以上の農地の利用集積等を達成できると見込まれる地域に対して償還金の利子相当額を一定期間助成
・土地改良区等に対して、災害により被災した農用地又は土地改良施設等の受益地に係る償還金の利子相当額を助成　等
　（補助率：定額）
2691 　対象基盤整備事業の実施地区において担い手への農用地の利用集積を促進するため、対象事業に係る農家負担金の軽減に資するよう、当該農家負担金について、（株）日本政策金融公庫等が土地改良区等に対し農家負担金の６分の５以内（但し、年度事業費の10％以内）に相当する額の無利子資金の貸付を行う。国は（株）日本政策金融公庫等に対して利子補給を行う。
（補助率：定額）
2692 国は重要無形民俗文化財以外の無形民俗文化財のうち、特に必要のあるものを選択し、自ら記録を作成できることになっている。この規定に基づき、「記録作成等の措置を講ずべき無形の民俗文化財」として選択した無形の民俗文化財のうち、複数の市町村にわたって広域的に伝承されていたり、保護団体が特定されていない祭りや年中行事については、その分布状況や伝承基盤が不明確なことから、地方公共団体等による記録の作成が進まない状況にあるため、特に変容・衰滅の恐れが高いものについて、計画的に映像等による記録化を進め、確実な記録保存を図る。
2693 　公益財団法人アイヌ文化振興・研究推進機構が行う、「アイヌ関連総合研究事業」、「アイヌ語の振興事業」、「アイヌ文化の振興事業」、「アイヌの伝統的生活空間再生事業」等に対して、補助（補助率：１／２）を行う。
　また、平成32年4月に北海道白老町に整備される「民族共生象徴空間」の中核施設となる「国立アイヌ民族博物館」の整備・運営準備のため、展示資料の調査・収集・保管、展示内容等の調査・検討、地元及びアイヌの人々など関係者等との調整を行う。
2694 　『アイヌ文化の復興等を促進するための民族共生象徴空間の整備及び管理運営に関する基本方針』（平成26年6月閣議決定/平成29年6月一部変更）並びに、「『民族共生の象徴となる空間』における博物館の整備・運営に関する調査検討委員会」において取りまとめられた「『民族共生の象徴となる

In [74]:
print(len(tf_business_list), len(word_list))
print(tf_business_list[0])
print(word_list)

4373 18135
ID                                                         15765
プロジェクトID                                                      16
公開年度                                                        2018
事業番号1                                                        nan
事業番号2                                                         11
事業番号3                                                        NaN
主要政策・施策                                                  ['その他']
府省庁                                                         内閣官房
事業名                                                       内閣広報経費
事業の目的          内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...
事業概要           内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...
執行額                                                           77
objective_x                                              -3.4392
objective_y                                              36.5031
Name: 0, dtype: object
['内閣総理大臣', 'が', '国際', '会議', '等', 'で', '外国', 'を', '訪問', '

In [75]:
words = tokenize(normalize_text(tf_business_list[0].loc['事業概要'])[0])

In [115]:
tf = {}
for data in tf_business_list:
  tf[data.loc["プロジェクトID"]] = {}
  words = tokenize(normalize_text(data.loc['事業概要']))
  tmp = {}
  for word in words:
    tmp[word] = words.count(word)/len(words)
  tf[data.loc["プロジェクトID"]] = tmp

In [ ]:
tf[4924]

In [ ]:
df_2018

In [131]:
len(word_list)

18135

In [132]:
import math
tmp = {}
for data in tf_business_list:
  words = tokenize(normalize_text(data.loc['事業概要']))
  check = []
  for word in words:
    if word not in check:
      check.append(word)
      if word not in tmp.keys():
        tmp[word] = 1
      else:
        tmp[word] += 1

In [ ]:
tmp

In [133]:
idf = {}
for word in word_list:
  idf[word] = math.log2((len(tf_business_list)-tmp[word]+0.5)/(tmp[word]+0.5))

In [ ]:
idf

In [134]:
sum = 0
for word in word_list:
  sum += tmp[word]

In [135]:
sum

249225

In [136]:
avgdl = sum/len(tf_business_list)

In [137]:
avgdl

56.99176766521838

In [140]:
okapi = {}
count = 0
for data in tf_business_list:
  if data.loc['プロジェクトID'] not in okapi.keys():
    okapi[data.loc['プロジェクトID']] = {}
  words = tokenize(normalize_text(data.loc['事業概要']))
  ex = {}
  # print(count, data.loc['プロジェクトID'], data.loc['事業名'], words)
  count += 1
  for word in words:
    ex[word] = idf[word] * ((tf[data.loc['プロジェクトID']][word]*(2.0+1)) / (tf[data.loc['プロジェクトID']][word] + 2.0 * (1-0.75+0.75*(tmp[word]/avgdl))))
  
  # print(ex)
  okapi[data.loc['プロジェクトID']] = ex

In [143]:
okapi[tf_business_list[0].loc['プロジェクトID']]

{'が': -9.280533919346896e-05,
 'こと': 0.0024347933216331986,
 'さ': 0.0019682212120036217,
 'し': -0.0010233750893053298,
 'する': -0.0009301360049168499,
 'せる': 0.026150739343417335,
 'た': -0.00021315052103970759,
 'ため': -0.0002887129068748907,
 'て': 0.0002374732185406405,
 'で': 0.00031723596830760486,
 'に': -0.005588387136640836,
 'の': -0.008497024395403403,
 'また': 0.003849473983756232,
 'や': 0.00026386438823890223,
 'やすく': 0.2717713787042636,
 'より': 0.030806550656725936,
 'を': -0.012249165795125014,
 'を通じて': 0.023985503825718534,
 'ホームページ': 0.12254321544657311,
 '会議': 0.026592417262917417,
 '伝える': 0.5178941985526625,
 '作成': 0.009389589268244293,
 '促進': 0.008042653320864991,
 '内閣': 0.7229392118896849,
 '内閣官房長官': 0.8877190166918318,
 '内閣総理大臣': 0.49081441030687434,
 '分かり': 0.3449967587834975,
 '動向': 0.05415012741403922,
 '動画': 0.5482159090527697,
 '同時通訳': 0.8877190166918318,
 '国民': 0.032811204945720415,
 '国際': 0.014847378882830175,
 '国際社会': 0.2485426809537909,
 '外国': 0.06644780749069275,
 '

In [142]:
max(okapi[tf_business_list[0].loc['プロジェクトID']])

'首相官邸'

In [95]:
for data in tf_business_list:
  max_okapi = data

0.006211180124223602

In [ ]:
count = 0
count_Okapi = {}
for data in similar_koyosha_list_in_business:
  for word in similar_koyosha:
    if word not in count_Okapi.keys():
      count_Okapi[word] = 0
    try:
      if okapi[data.loc['自治体コード']][data.loc['Ｎｏ']][data.loc['交付対象事業の名称']][word] >= 0.6:
        count_Okapi[word] += 1
      # print(data.loc['自治体コード'], data.loc['Ｎｏ'], data.loc['交付対象事業の名称'], word, okapi[data.loc['自治体コード']][data.loc['Ｎｏ']][data.loc['交付対象事業の名称']][word])
    except KeyError:
      count += 1
      if count % 1000000 == 0:
        print('running')

rank_Okapi = sorted(count_Okapi.items(), key=lambda x:-x[1])
print('end')

running
running
running
running
running
running
running
end


In [ ]:
rank_Okapi

In [ ]:
not_corona = []
for data in rank_Okapi:
  print(data)
  if data[1] >= 1:
    not_corona.append(data[0])

('住民税', 17)
('県庁', 14)
('期日前投票所', 14)
('ＤＶ', 14)
('県立高等学校', 14)
('新成人', 13)
('資料館', 13)
('老人福祉施設', 13)
('寄宿舎', 13)
('シェア', 12)
('出張所', 12)
('電話回線', 12)
('消防本部', 12)
('往診', 12)
('鏡', 12)
('コンピュータ', 12)
('ドアノブ', 12)
('成人', 11)
('サービスセンター', 11)
('噴霧器', 11)
('中継', 11)
('市議会', 11)
('サイクリング', 11)
('スポーツチーム', 11)
('布製マスク', 11)
('送風機', 10)
('歯', 10)
('洗濯', 10)
('サイクル', 10)
('診察室', 10)
('券売機', 10)
('斎場', 10)
('分署', 10)
('クレジットカード', 10)
('出先機関', 10)
('電子入札システム', 10)
('歯科医', 10)
('学園', 10)
('デイ', 10)
('老人福祉センター', 10)
('赤外線カメラ', 10)
('公用車', 9)
('広報誌', 9)
('指導者', 9)
('競技', 9)
('ボックス', 9)
('人工呼吸器', 9)
('外来患者', 9)
('読み聞かせ', 9)
('ペーパータオル', 9)
('キット', 9)
('パーク', 9)
('広域連合', 9)
('町長選挙', 9)
('農産物直売所', 9)
('問診', 9)
('浴室', 9)
('卸売市場', 9)
('家具', 9)
('午前', 9)
('友好', 9)
('式典', 9)
('専修学校', 9)
('成人式', 9)
('薬', 9)
('ＢＯＸ', 9)
('ビル', 9)
('呼び出し', 9)
('戸', 9)
('エア', 9)
('可燃ごみ', 9)
('封入', 9)
('アイソレーター', 9)
('光回線', 8)
('学習者', 8)
('口', 8)
('港湾', 8)
('位置情報', 8)
('環境整備事業', 8)
('トラック', 8)
('食堂', 8)
('タオル', 8)
('家屋', 8)
('

In [ ]:
not_corona_business = []
tmp = []
for data in similar_koyosha_list_in_business:
    words = tokenize(normalize_text(data.loc['事業概要（目的・効果）'])[0])
    for word in words:
      if (word in not_corona) and (data.loc['事業概要（目的・効果）'] not in tmp):
        tmp.append(data.loc['事業概要（目的・効果）'])
        not_corona_business.append(data)

with open('/content/drive/MyDrive/Colab Notebooks/pre-judgit/コロナ/words_csv/not_corona_business.csv', 'w', encoding='cp932')as f:
  writer = csv.writer(f)
  for data in not_corona_business:
    writer.writerow(data)

In [ ]:
len(not_corona_business)

6729

In [ ]:
tmp = []
not_corona_business = []
size_area={}
for i in range(len(json_list)):
  print('running')
  with open(json_list[i], encoding='utf8')as f:
    df = pd.read_json(f, lines = True)

  sum = 0  
  for j in range(len(df)):
    if df.loc[j]['事業概要（目的・効果）'] != '':
      words = tokenize(normalize_text(df.loc[j]['事業概要（目的・効果）'])[0])
      for word in words:
        if (word in not_corona) and (df.loc[j]['事業概要（目的・効果）'] not in tmp):
          tmp.append(df.loc[j]['事業概要（目的・効果）'])
          not_corona_business.append(df.loc[j])
          sum += 1
  size_area[keys[i]] = sum

print('end')

running
running
running
running
running
running
running
running
end


In [ ]:
for key in keys:
 print(size_area[key])

820
937
984
1090
844
500
388
1166


In [ ]:
not_corona

['住民税',
 '県庁',
 '期日前投票所',
 'ＤＶ',
 '県立高等学校',
 '新成人',
 '資料館',
 '老人福祉施設',
 '寄宿舎',
 'シェア',
 '出張所',
 '電話回線',
 '消防本部',
 '往診',
 '鏡',
 'コンピュータ',
 'ドアノブ',
 '成人',
 'サービスセンター',
 '噴霧器',
 '中継',
 '市議会',
 'サイクリング',
 'スポーツチーム',
 '布製マスク',
 '送風機',
 '歯',
 '洗濯',
 'サイクル',
 '診察室',
 '券売機',
 '斎場',
 '分署',
 'クレジットカード',
 '出先機関',
 '電子入札システム',
 '歯科医',
 '学園',
 'デイ',
 '老人福祉センター',
 '赤外線カメラ',
 '公用車',
 '広報誌',
 '指導者',
 '競技',
 'ボックス',
 '人工呼吸器',
 '外来患者',
 '読み聞かせ',
 'ペーパータオル',
 'キット',
 'パーク',
 '広域連合',
 '町長選挙',
 '農産物直売所',
 '問診',
 '浴室',
 '卸売市場',
 '家具',
 '午前',
 '友好',
 '式典',
 '専修学校',
 '成人式',
 '薬',
 'ＢＯＸ',
 'ビル',
 '呼び出し',
 '戸',
 'エア',
 '可燃ごみ',
 '封入',
 'アイソレーター',
 '光回線',
 '学習者',
 '口',
 '港湾',
 '位置情報',
 '環境整備事業',
 'トラック',
 '食堂',
 'タオル',
 '家屋',
 'サーバ',
 '支署',
 'ホームゲーム',
 'ＰＡＹＰＡＹ',
 '法務局',
 '箱',
 'ドーム',
 '庭園',
 '排気',
 'プロジェクター',
 '仙台市',
 '練習',
 '携帯',
 '近距離',
 '自然公園',
 '照射',
 '公衆トイレ',
 'テラス',
 '予備',
 '移設',
 '包装',
 '液体',
 '水際対策',
 '病室',
 '選手',
 '育児支援',
 '除',
 '東北',
 'ＶＲ',
 '浸水',
 '耳',
 'ライセンス',
 '２人',
 '硬貨',
 '宮城県',
 '国内線',
 '帳',
 '保冷

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/pre-judgit/コロナ/words_csv/rank_okapi.csv', 'w', encoding='cp932')as f:
  writer = csv.writer(f)
  for data in rank_Okapi:
    if data[1] >= 1:
      writer.writerow(data)

In [ ]:
tf[1204]['庁舎感染予防対策（庁舎管理費）']

{'する': 0.041666666666666664,
 'ため': 0.041666666666666664,
 'に': 0.125,
 'の': 0.08333333333333333,
 'や': 0.041666666666666664,
 'アクリル': 0.041666666666666664,
 'パーテーション': 0.041666666666666664,
 '係る': 0.041666666666666664,
 '内': 0.041666666666666664,
 '対策': 0.041666666666666664,
 '市庁舎': 0.041666666666666664,
 '市有': 0.041666666666666664,
 '施設': 0.041666666666666664,
 '物品': 0.041666666666666664,
 '窓口': 0.041666666666666664,
 '等': 0.041666666666666664,
 '製': 0.041666666666666664,
 '設置': 0.041666666666666664,
 '購入': 0.041666666666666664,
 '防止': 0.041666666666666664,
 '飛沫感染': 0.041666666666666664}

In [ ]:
for data in similar_koyosha_list_in_business:
  if data.loc['交付対象事業の名称'] == '庁舎感染予防対策（庁舎管理費）':
    print(data.loc['自治体コード'], data.loc['交付対象事業の名称'])

1204 庁舎感染予防対策（庁舎管理費）
1204 庁舎感染予防対策（庁舎管理費）


In [ ]:
test = {}
test['漁協経営継続緊急支援事業費'] = {'abc':'123'}

In [ ]:
test

{'漁協経営継続緊急支援事業費': {'abc': '123'}}